In [1]:
from __future__ import print_function
import StringIO
import random

In [2]:
class Player:
    
    def __init__(self, name, symbol, strategy):
        self._name = name
        self._symbol = symbol
        self._strategy = strategy
    
    def choose(self, board):
        return self._strategy.next_move(board=board)
    
    def symbol(self):
        return self._symbol
    
    def name(self):
        return self._name
    
    def __eq__(self, other):
        return other is not None and self._name == other._name and self._symbol == other._symbol
    
    def __hash__(self):
        return hash((self._name, self._symbol))
    
    def __repr__(self):
        return "{} ({})".format(self._name, self._symbol)

In [3]:
class Board:
    
    _EMPTY_SYMBOL = ' '
    _ROWS = 3
    _COLUMNS = 3
    _SIZE = _ROWS * _COLUMNS
    
    def __init__(self):
        self._board = [self._EMPTY_SYMBOL] * self._SIZE
    
    def __repr__(self):
        return ''.join([" {} {}".format(self._board[i], self._repr_board(i)) for i in range(self._SIZE)])
    
    def _repr_board(self, i):
        return "|" if i % 3 != 2 else "\n{}\n".format("-" * (self._SIZE + 2) if i /3 != 2 else "")
        
    def update(self, row, col, player):
        self._board[row * self._ROWS + col] = player.symbol()
    
    def check_coordinate(self, row, col):
        return self._board[row * self._ROWS + col]
        
    def free_coordinates(self):
        return [(i / 3, i % 3) for i in range(self._SIZE) if self._board[i] == self._EMPTY_SYMBOL]
        
    def is_empty(self):
        return len(self.free_coordinates()) == self._SIZE
    
    def is_full(self):
        return len(self.free_coordinates()) == 0
    
    def __eq__(self, other):
        return self._board == other._board
    
    def __hash__(self):
        return hash(''.join(self._board))
    
    def show(self):
        print(self.__repr__())
        
    def clone(self):
        clone = Board()
        clone._board = [self._board[i] for i in range(self._SIZE)]
        return clone

In [4]:
class RandomStrategy:

    def next_move(self, board):
        free_coordinates = board.free_coordinates()
        return free_coordinates[random.randint(0, len(free_coordinates) - 1)] if len(free_coordinates) > 0 else None

In [5]:
class HumanStrategy:

    def next_move(self, board):
        choice_str = raw_input("Your turn (ex: 1,2)>")
        coordinates_str = choice_str.split(",")
        return tuple(map(int, coordinates_str))

In [6]:
class TicTacToeState:
    
    def __init__(self, board, reward):
        self._board = board
        self._reward = reward
        self._previous = []
        self._next = []
    
    def board(self):
        return self._board
    
    def reward(self):
        return self._reward
    
    def next_states(self):
        return self._next
    
    def update_reward(self, new_reward):
        self._reward = new_reward
    
    def add_next_state(self, state):
        self._next.append(state)
    
    def add_previous_state(self, state):
        self._previous.append(state)

In [7]:
class ResultChecker:
    
    def __init__(self, player_1, player_2):
        self._player_1 = player_1
        self._player_2 = player_2
    
    def check_winner(self, board):
        rows = {i: [] for i in range(3)}
        cols = {i: [] for i in range(3)}
        diag = []
        rdiag = []
        
        for i in range(9):
            row = i / 3
            col = i % 3
            rows[row].append(board.check_coordinate(row, col))
            cols[col].append(board.check_coordinate(row, col))
            if row == col:
                diag.append(board.check_coordinate(row, col))
            if row + col == 2:
                rdiag.append(board.check_coordinate(row, col))
        
        for candidate in rows.values() + cols.values() + [diag, rdiag]:
            for player in [self._player_1, self._player_2]:
                if candidate == [player.symbol()] * 3:
                    return player
                
        return None

In [10]:
class ReinforcementLearningStrategy:
    
    def __init__(self, explore=0.15, alpha=0.06):
        self._explore = explore
        self._alpha = alpha
        self._state_map = dict()
        self._choices = []
        self._target_player = None
        
    def new_match(self):
        self._choices = []

    def generate_all_states(self, player_1, player_2, target_player):
        self._target_player = target_player
        result_checker = ResultChecker(player_1, player_2)
        
        initial_state = TicTacToeState(board=Board(), reward=0)
        self._state_map[initial_state.board()] = initial_state
        stack = [(initial_state, player_1), (initial_state, player_2)]
        
        while len(stack) > 0:            
            cur_state, cur_player = stack.pop()
            cur_board = cur_state.board()
            
            if result_checker.check_winner(cur_board):
                continue
            
            for row, col in cur_board.free_coordinates():
                new_board = cur_board.clone()
                new_board.update(row, col, cur_player)
                
                new_state = None
                
                if new_board in self._state_map:
                    new_state = self._state_map[new_board]
                else:
                    winner = result_checker.check_winner(new_board)
                    
                    if winner == target_player:
                        reward = 1.0
                    elif winner is not None: # Player 2 won
                        reward = 0.0
                    elif len(new_board.free_coordinates()) == 0: # tie
                        reward = 0.0
                    else:
                        reward = 0.5
                        
                    new_state = TicTacToeState(board=new_board, reward=reward)
                    self._state_map[new_state.board()] = new_state
                    
                new_state.add_previous_state(cur_state)
                cur_state.add_next_state(new_state)                
                stack.append((new_state, player_1 if cur_player != player_1 else player_2))
                
    def next_move(self, board):
        next_state, to_update = self._chose_next_state(board)
        
        if len(self._choices) > 0 and to_update:
            self._update_reward(prev_state=self._choices[-1], next_state=next_state)
            
        self._choices.append(next_state)
        chosen_move = set(board.free_coordinates()) - set(next_state.board().free_coordinates())
        return chosen_move.pop() if len(chosen_move) > 0 else None
        
    
    def _chose_next_state(self, board):
        free_coordinates = board.free_coordinates()
        cur_state = self._state_map[board]
        
        if len(free_coordinates) == 0 or len(cur_state.next_states()) == 0:
            return cur_state, True
        elif random.random() < self._explore: # Exploring
            choice = free_coordinates[random.randint(0, len(free_coordinates) - 1)]
            new_board = board.clone()
            new_board.update(choice[0], choice[1], self._target_player)
            return self._state_map[new_board], False
        else: # Exploiting
            state_with_max_reward = max(cur_state.next_states(), key=lambda s: s.reward())
            return state_with_max_reward, True
    
    def _update_reward(self, prev_state, next_state):
        new_reward = prev_state.reward() + self._alpha * (next_state.reward() - prev_state.reward())
        # print("Updating reward from {} to: {}".format(prev_state.reward(), new_reward))
        prev_state.update_reward(new_reward)

In [11]:
class Match:
    
    def __init__(self, board, player_1, player_2):
        self._board = board
        self._player_1 = player_1
        self._player_2 = player_2
        self._result_checker = ResultChecker(player_1, player_2)
    
    def play(self, show=True):
        
        if not self._board.is_empty():
            raise Exception("Board is not empty!\n{}".format(self._board))
        
        cur_player = self._select_first_player(show)
        while not self._board.is_full():
            choise = cur_player.choose(board=board)
            if show:
                print("Player {} ({}) has chosen {}".format(cur_player.name(), cur_player.symbol(), choise))
            self._board.update(choise[0], choise[1], cur_player)
            if show:
                self._board.show()
            winner = self._result_checker.check_winner(self._board)
            if winner:
                break
            cur_player = self._player_1 if cur_player == self._player_2 else self._player_2
        
        winner = self._result_checker.check_winner(self._board)
        if winner:
            if show:
                print("Player {} ({}) won the game! :) ".format(winner.name(), winner.symbol()))
        else:
            if show:
                print("It's a tie :D")
        
        player_1.choose(board=board)
        player_2.choose(board=board)
        return winner
    
    def _select_first_player(self, show):
        if show:
            print("Flipping a coin to decide which player will start the match...")
        first_player = self._player_1 if random.randint(0, 1) == 0 else self._player_2
        if show:
            print("{} will start the game".format(first_player.name()))
        return first_player
    

In [12]:
rls_1 = ReinforcementLearningStrategy(explore=0.30, alpha=0.05)
rls_2 = ReinforcementLearningStrategy(explore=0.40, alpha=0.10)

In [13]:
player_1 = Player(name='first', symbol='X', strategy=rls_1)
player_2 = Player(name='second', symbol='O', strategy=rls_2)

In [14]:
time rls_1.generate_all_states(player_1=player_1, player_2=player_2, target_player=player_1)

CPU times: user 50 s, sys: 200 ms, total: 50.2 s
Wall time: 50.2 s


In [15]:
time rls_2.generate_all_states(player_1=player_1, player_2=player_2, target_player=player_2)

CPU times: user 50.5 s, sys: 201 ms, total: 50.7 s
Wall time: 50.7 s


In [16]:
len(rls_1._state_map)

8533

In [32]:
matches = 1000000

results = dict()

for i in range(matches):
    if i % 100000 == 0:
        print("MATCH: {}".format(i))
    board = Board()
    rls_1.new_match()
    rls_2.new_match()
    match = Match(board=board, player_1=player_1, player_2=player_2)
    winner = match.play(show=False)
    if winner:
        if winner.name() not in results:
            results[winner.name()] = 1
        results[winner.name()] += 1

MATCH: 0
MATCH: 100000
MATCH: 200000
MATCH: 300000
MATCH: 400000
MATCH: 500000
MATCH: 600000
MATCH: 700000
MATCH: 800000
MATCH: 900000


In [33]:
results

{'first': 530560, 'second': 405647}

In [34]:
human_player = Player(name='William', symbol='O', strategy=HumanStrategy())

In [42]:
board = Board()
rls_1.new_match()
rls_2.new_match()
match = Match(board=board, player_1=player_1, player_2=human_player)
match.play()

Flipping a coin to decide which player will start the match...
first will start the game
Player first (X) has chosen (1, 1)
   |   |   
-----------
   | X |   
-----------
   |   |   


Your turn (ex: 1,2)>2,1
Player William (O) has chosen (2, 1)
   |   |   
-----------
   | X |   
-----------
   | O |   


Player first (X) has chosen (2, 2)
   |   |   
-----------
   | X |   
-----------
   | O | X 


Your turn (ex: 1,2)>0,0
Player William (O) has chosen (0, 0)
 O |   |   
-----------
   | X |   
-----------
   | O | X 


Player first (X) has chosen (1, 2)
 O |   |   
-----------
   | X | X 
-----------
   | O | X 


Your turn (ex: 1,2)>0,2
Player William (O) has chosen (0, 2)
 O |   | O 
-----------
   | X | X 
-----------
   | O | X 


Player first (X) has chosen (1, 0)
 O |   | O 
-----------
 X | X | X 
-----------
   | O | X 


Player first (X) won the game! :) 


first (X)